Gates for qubitizing the second quantized chemistry Hamiltonian in the plane-wave dual basis.

This follows section V. of the [Linear T Paper](https://arxiv.org/abs/1805.03662).

Here we implement SelectChem (Fig. 14.), SubPrepareChem (Fig. 15) and
PrepareChem (Fig. 16), which are specializations of the ab-initio Hamiltonian
for the case of a plane wave (dual) basis.

Under the Jordan-Wigner transformation the Hamiltonian is given by

$$
\def\Zvec{\overrightarrow{Z}}
\def\hop#1{#1_{p,\sigma} \Zvec #1_{q,\sigma}}
H = \sum_{p\ne q} \frac{T(p-q)}{2}(\hop{X}+\hop{Y})
+ \sum_{(p\alpha)\ne (q\beta)} \frac{V(p-q)}{4} Z_{p\alpha}Z_{q\beta}
- \sum_{p\sigma} \frac{T(0) + U(p) + \sum_q V(p-q)}{2} Z_{p\sigma}
+ \sum_{p}\left(T(0) + U(p) + \sum_q \frac{V(p-q)}{2}\right)\mathbb{1},
$$

Note that in the above expression $p$ is really a three-dimensional vector of
integers with each component $p_i \in [0, M-1]$ where $M$ is the
related to the real-space grid resolution and is expected as input by the user.
In total there are $N=2M^3$ spin-orbitals.  In what follows we often label
registers with a single label $(e.g. |p\rangle)$, but this should be understood
to mean $|\vec{p}\rangle = |p_x\rangle|p_y\rangle|p_z\rangle$ with each register
of size $\log M$.  This model can be constructed using the functions provided in
`openfermion.hamiltonians.plane_wave_hamiltonian`.


This model consists of a PREPARE and SELECT operation where our selection operation has indices
for $p$, $\alpha$, $q$, and $\beta$ as well as two indicator bits $U$ and $V$. There are four cases
considered in both the PREPARE and SELECT operations corresponding to the terms in the Hamiltonian:

 - $U = 1, V = 0$ and $(p\alpha) = (q\beta)$, single-body $Z$
 - $U = 0, V=1$, $(p\alpha) \ne (q\beta)$, spin-spin $ZZ$ term
 - $U = 0, V = 0, p < q \wedge (\alpha = \beta)$, $XZX$ term
 - $U = 0, V = 0, p > q \wedge (\alpha = \beta)$, $YZY$ term

See the documentation for `PreparePWDual` and `SelectPWDual` for more details.

In [ ]:
import numpy as np
import cirq_ft.infra.testing as cf_testing
from cirq_ft.infra.t_complexity_protocol import t_complexity
from cirq_ft.infra.jupyter_tools import display_gate_and_compilation
from typing import *
import matplotlib.pyplot as plt

# SelectPWDual

The SELECT operation optimized for the plane wave dual basis Hamiltonian

#### Args:
- `num_spin_orb`: the number of spin-orbitals.
- `control_val`: Optional bit specifying the control value for constructing a controlled
        version of this gate. Defaults to None, which means no control.

#### Registers:
- `control`: A control bit for the entire gate.
- `theta`: Whether to apply a negative phase.
- `UV`: Flag to apply parts of Hamiltonian. See module docstring.
- `p`: Spatial orbital index.
- `alpha`: Spin index for orbital p.
- `q`: Spatial orbital index.
- `beta`: Spin index for orbital q.
- `target`: The system register to apply the select operation.

#### References:
- Section V. Eq. 44 and Fig. 14 of https://arxiv.org/abs/1805.03662.

In [ ]:
from cirq_ft.algos.chemistry import SelectPWDual

g = cf_testing.GateHelper(
    SelectPWDual(2, 1)
)

display_gate_and_compilation(g)

### Complexity

According to Fig. 14 of the Linear T paper the Select gate should have complexity scaling like $12 N + O(\log N)$. Let's check that this is the case.

In [ ]:
# First define some utility fitting functions which will be used elsewhere in the notebook
from numpy.typing import NDArray
import scipy.optimize

def linear(x: NDArray, a: float, b: float) -> NDArray:
    return a*x + b

def fit_linear(x: NDArray, y: NDArray, p0=None) -> Tuple[float, float]:
    popt, _ = scipy.optimize.curve_fit(linear, x, y, p0=p0)
    return popt 


In [ ]:
t_comp = []
M_values = np.arange(4, 12, 2)
N_values = 2 * M_values**3 # spin orbitals

for M  in M_values:
    num_spatial = M**3
    Us, Ts, Vs, Vxs = np.random.normal(size=4 * num_spatial).reshape((4, num_spatial))
    sp = SelectPWDual(int(M), 1)
    t_comp.append(t_complexity(sp).t)

plt.plot(N_values, t_comp, marker="o", lw=0)
a, b = fit_linear(N_values, t_comp)
xs = [N_values[0], N_values[-1]]
ys = [linear(xs[0], a, b), linear(xs[1], a, b)]
plt.plot(xs, ys, ls=":", label=f"y = {a:3.2f} $N$ + {b:3.2f}")
plt.xlabel("$N$")
plt.legend()
plt.ylabel("$T$ count")

Success!

## `SubPrepareChem`
Sub-prepare circuit for the plane wave dual Hamiltonian.

This circuit acts on a state like:

$$
    \mathrm{SUBPREPARE}|0\rangle^{\otimes{2 + \log N}} \rightarrow
    \sum_d^{N-1}\left(\tilde{U}(d)|\theta_d\rangle|1\rangle_U|0\rangle_V
    +\tilde{T}(d)|\theta_d^{(0)}\rangle|0\rangle_U|0\rangle_V
    +\tilde{V}(d)|\theta_d^{(1)}\rangle|0\rangle_U|1\rangle_V
    \right)|d\rangle
$$

where

$$
    \tilde{U}(p) = \sqrt{\frac{Q(p)}{2\lambda}}\\
    \tilde{T}(p) = \sqrt{\frac{T(p)}{\lambda}}\\
    \tilde{V}(p) = \sqrt{\frac{V(p)}{4\lambda}}\\
    \theta_p = \frac{1-\mathrm{sign}(-Q(p))}{2}\\
    \theta_p^{(0)} = \frac{1-\mathrm{sign}(T(p))}{2}\\
    \theta_p^{(1)} = \frac{1-\mathrm{sign}(V(p))}{2}\\
$$
and
$$
    Q(p) = |T(0)+U(p)+V_x(p)|\\
    V_x(p) = \sum_q V(p-q)
$$

#### Parameters
 - `num_spin_orbs`: number of spin orbitals (typically called N in literature.)
 - `theta`: numpy array of theta values of shape [3, M, M, M]
 - `altUV`: numpy array of alternate U values of shape [3, M, M, M]
 - `altpxyz`: numpy array of alternate indices p = (px,py,pz) values of shape [3, M, M, M]
 - `keep`: numpy array of keep values of shape [3, M, M, M]
 - `mu`: Exponent to divide the items in keep by in order to obtain a
 - `probability. See `preprocess_lcu_coefficients_for_reversible_alias_sampling`.`:  

Registers:
    theta: Whether to apply a negative phase.
    U: Flag to apply parts of Hamiltonian. See module docstring.
    V: Flag to apply parts of Hamiltonian. See module docstring.
    p: (vector) Spatial orbital index, p = (px, py, pz)

#### References
Section V. and Fig. 14 of https://arxiv.org/abs/1805.03662. 

#### References
[Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Section III.D. and Figure 15. Note there is an error in the circuit diagram in the paper (there should be no data: $\theta_{alt_l}$ gate in the QROM load.)


In [ ]:
from cirq_ft.algos.chemistry import SubPreparePWDual

M = 2
num_spatial = M**3
Us, Ts, Vs, Vxs = np.random.normal(size=4 * num_spatial).reshape((4, num_spatial))
sp = SubPreparePWDual.build_from_coefficients(Ts, Us, Vs, Vxs, probability_epsilon=0.01)
g = cf_testing.GateHelper(
    sp
)

display_gate_and_compilation(g)

### Complexity Check

According to Fig. 15 of the Linear T paper, the SubPrepare circuit (our SubPreparePWDual) should have complexity that scales like $6N + O(\mu + \log N)$. Let's check this is the case.

In [ ]:
from cirq_ft.infra.t_complexity_protocol import t_complexity
import matplotlib.pyplot as plt
t_comp = []
M_values = np.arange(4, 12, 2)
N_values = 2 * M_values**3 # spin orbitals

for M  in M_values:
    num_spatial = M**3
    Us, Ts, Vs, Vxs = np.random.normal(size=4 * num_spatial).reshape((4, num_spatial))
    sp = SubPreparePWDual.build_from_coefficients(Ts, Us, Vs, Vxs, probability_epsilon=1e-16)
    t_comp.append(t_complexity(sp).t)

plt.plot(N_values, t_comp, marker="o", lw=0)
a, b = fit_linear(N_values, t_comp)
xs = [N_values[0], N_values[-1]]
ys = [linear(xs[0], a, b), linear(xs[1], a, b)]
plt.plot(xs, ys, ls=":", label=f"y = {a:3.2f} $N$ + {b:3.2f}")
plt.xlabel("$N$")
plt.legend()
plt.ylabel("$T$ count")

Success!

## `PreparePWDual`
Prepare circuit for the plane wave dual Hamiltonian.

This circuit acts on a state like:

$$
    \mathrm{PREPARE}|0\rangle^{\otimes{3 + 2log N}} \rightarrow
    \sum_{p\sigma} \left(
    \tilde{U}(p)|\theta_p|1\rangle_U|0\rangle_V|p\alpha q\beta\rangle
    +\tilde{T}(p-q)|\theta_{p-q}^{(0)}\rangle|0\rangle_U|0\rangle_V |p\alpha q\beta\rangle
    +\tilde{V}(p-q)|\theta_{p-q}^{(1)}\rangle|0\rangle_U|1\rangle_V |p\alpha q\beta\rangle
    \right)
$$

See SubPrepareChem docstring for definitions of terms.

#### Parameters
- `M`: number of grid points in each dimension.
- `T`: kinetic energy matrix elements of shape (M, M, M).
- `U`: external potential matrix elements of shape (M, M, M).
- `V`: electron-electron interaction matrix elements of shape (M, M, M).
- `Vx`: "Exchage potential" matrix $\sum_q V(p-q)$ of shape (M, M, M).
- `probability_epsilon`: The epsilon that we use to set the precision of of the
    subprepare approximation. This parameter is called mu in the linear T paper.

### Registers:
- `theta`: Whether to apply a negative phase.
- `U`: Flag to apply parts of Hamiltonian. See module docstring.
- `V`: Flag to apply parts of Hamiltonian. See module docstring.
- `p`: Spatial orbital index.
- `alpha`: Spin index for orbital p.
- `q`: Spatial orbital index.
- `beta`: Spin index for orbital q.

#### References
[Encoding Electronic Spectra in Quantum Circuits with Linear T Complexity](https://arxiv.org/abs/1805.03662). Babbush et. al. (2018). Section III.D. and Figure 15. Note there appears to be an error in the circuit diagram in the paper (there should be no data: $\theta_{alt_l}$ gate in the QROM load. In addition the Indexed add $\vec{q} \mod M$ gate should just be a modular addition (no unary iteration).)


In [ ]:
from cirq_ft.algos.chemistry import PreparePWDual

M = 2
num_spatial = M**3
Us, Ts, Vs, Vxs = np.random.normal(size=4 * num_spatial).reshape((4, num_spatial))

g = cf_testing.GateHelper(
    PreparePWDual(M=M, T=Ts, U=Us, V=Vs, Vx=Vxs)
)

display_gate_and_compilation(g)

### Complexity Check

According to Fig. 16 of the Linear T paper, the Prepare circuit (our PreparePWDual) should have complexity that scales like $6N + O(\mu + \log N)$, and is dominated by the SupPreparePWDual block. Let's check that this is the case.

In [ ]:
from cirq_ft.infra.t_complexity_protocol import t_complexity
import matplotlib.pyplot as plt
t_comp = []
M_values = np.arange(4, 12)
N_values = 2 * M_values**3 # spin orbitals

for M  in M_values:
    num_spatial = M**3
    Us, Ts, Vs, Vxs = np.random.normal(size=4 * num_spatial).reshape((4, num_spatial))
    sp = PreparePWDual(int(M), Ts, Us, Vs, Vxs, probability_epsilon=1e-8)
    t_comp.append(t_complexity(sp).t)

plt.plot(N_values, t_comp, marker="o", lw=0)
a, b = fit_linear(N_values, t_comp)
xs = [N_values[0], N_values[-1]]
ys = [linear(xs[0], a, b), linear(xs[1], a, b)]
plt.plot(xs, ys, ls=":", label=f"y = {a:3.2f} $N$ + {b:3.2f}")
plt.xlabel("$N$")
plt.legend()
plt.ylabel("$T$ count")